In [1]:
#データの読み込み
import numpy as np
import pandas as pd

data = pd.read_csv('titanic_train.csv')
df = pd.DataFrame(data)
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
#欠損値の確認
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [3]:
#乗客IDと名前とTicketとCabin以外を切り出す
#Cabinは欠損値が多すぎるので削除
#それ以外は生存と関係がなさそうなので削除
df1 = df.iloc[:,[2,4,5,6,7,9,11]]

df1.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
3,1,female,35.0,1,0,53.1000,S
4,3,male,35.0,0,0,8.0500,S


In [4]:
#マッピングを行う
emb = pd.get_dummies(df1['Embarked'])
df2 = pd.concat([df1.iloc[:,0:6],emb], axis=1)
sex_map = {'male': 0, 'female': 1}
df2['Sex'] = df2['Sex'].map(sex_map)

df2.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,C,Q,S
0,3,0,22.0,1,0,7.2500,0,0,1
1,1,1,38.0,1,0,71.2833,1,0,0
2,3,1,26.0,0,0,7.9250,0,0,1
3,1,1,35.0,1,0,53.1000,0,0,1
4,3,0,35.0,0,0,8.0500,0,0,1


In [5]:
#Ageに関する欠損値処理（平均値補完）
df2['Age'] = df2['Age'].fillna(df2['Age'].mean())
df2.isnull().sum()

Pclass    0
Sex       0
Age       0
SibSp     0
Parch     0
Fare      0
C         0
Q         0
S         0
dtype: int64

In [6]:
#Age, Fareを標準化
#特徴量をarrayにする
from sklearn.preprocessing import StandardScaler

age_fare = df2[['Age', 'Fare']].values
ss = StandardScaler()
age_fare_std = ss.fit_transform(age_fare)
X = np.hstack([df2.iloc[:,[0,1,3,4,6,7,8]].values, age_fare_std])
y = df['Survived'].values
X[:5]

array([[ 3.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         0.        ,  1.        , -0.5924806 , -0.50244517],
       [ 1.        ,  1.        ,  1.        ,  0.        ,  1.        ,
         0.        ,  0.        ,  0.63878901,  0.78684529],
       [ 3.        ,  1.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  1.        , -0.2846632 , -0.48885426],
       [ 1.        ,  1.        ,  1.        ,  0.        ,  0.        ,
         0.        ,  1.        ,  0.40792596,  0.42073024],
       [ 3.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  1.        ,  0.40792596, -0.48633742]])

In [7]:
#ホールド・アウト法
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [8]:
#SVM
from sklearn.svm import SVC
svc = SVC(kernel='linear')
svc.fit(X_train, y_train)

print(svc.score(X_train, y_train))
print(svc.score(X_test, y_test))

0.786516853933
0.787313432836


In [9]:
#ロジスティック回帰
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(C=0.1)
lr.fit(X_train, y_train)

print(lr.score(X_train, y_train))
print(lr.score(X_test, y_test))

0.807383627608
0.813432835821
